In [215]:
import python_speech_features as sf
import scipy
import scipy.io.wavfile
import scipy.cluster
import sys
import os
import glob
import numpy as np
import json
import itertools
import cPickle as cp

In [150]:
(f,s)=scipy.io.wavfile.read('/home/nishant/Desktop/recordings/0_nicolas_19.wav')
print 'sampling freq is:%d kHZ'%(f/1000)

sampling freq is:8 kHZ


### details about mfcc and their superiority along with code and library have been explained in the document

**function to calculate mfcc for each of the audio files**

**You can see clearly here the data non uniformities I observed ,data by speaker jason is clearly useless,very long,and wrong with respect to its label.also the user nicolas data has two dimensions,on careful observation,I found it basically has tuples with the same values,so cleaning it to use only one of the tuple elements was necessary**

In [221]:
f1,s1=scipy.io.wavfile.read('/home/nishant/Desktop/recordings/0_nicolas_19.wav')
f2,s2=scipy.io.wavfile.read('/home/nishant/Desktop/recordings/0_jackson_0.wav')
f3,s3=scipy.io.wavfile.read('/home/nishant/Desktop/recordings/0_jason_12.wav')
f4,s4=scipy.io.wavfile.read('/home/nishant/Desktop/recordings/0_theo_34.wav')
print s1.shape,s2.shape,s3.shape,s4.shape

 (3688, 2) (5148,) (23999,) (4231,)


In [207]:
def findmfcc(s,fs):
    a=[]
    #code for silence removal,was volatile and not very effective only increasing runtime,hence commented
    #this was based on the trimmer.py provided with code
    
    #for idx, point in enumerate(s[:]):
            #if abs(point) > 150:
             #  start = idx
             #  break
    #for idx, point in enumerate(s[::-1]):
          #  #if abs(point) > 150:
               # end = len(s) - idx
                #break
    #s=s[start:end]
    a=sf.mfcc(s,fs)
    a=scipy.cluster.vq.whiten(np.asarray(a))
    a=scipy.cluster.vq.kmeans(a,1)
    return a[0]

**converting each audio file to mfcc vectors along with one hot encoded labels,while discarding all the jason files**

In [208]:
def create_data():
    path='../nishant/Desktop/digit_recognition/Data/recordings/'
    features=[]
    labels=[]

    for i in range(10):
        
        files = glob.glob(path+str(i)+'*')
        #print files
        for f in files:
            filename = os.path.basename(f)
            if filename[2:7]=='jason':
                continue
            fs,sound=scipy.io.wavfile.read(f)
            
            if len(sound.shape)==2:
                sound=np.array([k[0] for k in sound])
            mfcc_vect=findmfcc(sound,fs)
            features.append(mfcc_vect)
            label = np.zeros(10)
            label[i] = 1.0
            labels.append(label)
    features = list(itertools.chain.from_iterable(features))
    labels=np.array(labels)
    features=np.array(features)
    
    return features,labels
    
    
    

In [218]:
path='../nishant/Desktop/digit_recognition/Data/recordings/'
newpath='../nishant/Desktop/digit_recognition/features'
x_train,y_train=create_data()
print x_train
if not os.path.isdir(newpath):
    os.makedirs(newpath)
dat=[x_train,y_train]
with open(newpath+'/data.p','wb') as f:
    cp.dump(dat,f)

[[  9.17359334   1.52348152  -0.02796871 ...,  -0.23122445  -0.45300224
   -1.11229628]
 [  8.41724663   1.50730392  -0.25423204 ...,  -0.33492478  -0.22269325
   -0.81681781]
 [  8.35676396   1.23340277  -0.22144965 ...,   0.16379507  -0.20743994
   -1.43899699]
 ..., 
 [ 15.81513847   1.86393871   0.24996151 ...,   1.23432014   1.35190087
    0.07747606]
 [  7.58431043  -0.09793268  -0.47481588 ...,  -0.75819508  -1.81654454
    0.68403551]
 [ 16.18351063   2.07356315   0.30892168 ...,   1.01224972   1.20670062
    0.27163475]]
